In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
from statsmodels.datasets import co2, sunspots
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import acf, pacf

In [2]:
def drop_nan(df):
    threshold = round(len(df) / 4)
    df.dropna(axis=1, inplace=True, thresh=threshold)
    df.dropna(axis=0, inplace=True)

# Esercizio n.ro 1

Effettuare la decomposizione STL sul dataset `sunspots`, e la modellazione con ARIMA sul dataset `co2`.

Articolare il processing nei seguenti step.

1. Caricare i dataset utilizzando il modello `load_pandas()`.
2. Effettuare la decomposizione STL sul dataset `sunspots` mediante il metodo `STL` di Statsmodels.
3. Studiare le funzioni ACF e PACF per il dataset `co2` mediante le funzioni `plot_acf` e `plot_pacf`.
4. Provare a determinare di conseguenza gli ordini ottimali per i processi AR, I ed MA di un ARIMA, motivando la scelta. Qualora l'analisi della ACF e della PACF non permetta di determinare facilmente gli ordini dei processi AR, I ed MA, provare ad utilizzare un approccio di tipo *trial and error*, stimando i valori migliori per suddetti parametri.

Commentare i risultati ottenuti.

# Esercizio n.ro 2

Utilizzare le tecniche del *deep learning* per creare una rete neurale in grado di classificare i passeggeri del Titanic in base al fatto che siano sopravvissuti o meno.

Articolare il processing nei seguenti step.

1. Caricare il dataset usando il metodo `fetch_openml('titanic', version=1, as_frame=True)`.
2. Rimuovere i dati univoci (ad esempio, nome e numero di biglietto) utilizzando i metodi messi a disposizione da Pandas.
3. Eliminare i valori `NaN` e `None` dal dataframe. Prediligere, se possibile, l'eliminazione dei singoli dati.
4. Creare un modello di rete neurale usando la Sequential API. La rete dovrà avere tre layer completamente connessi (`Dense`) ed un layer di classificazione finale. In particolare, i primi tre layer dovranno avere 16 neuroni, con funzione di attivazione `relu`, mentre l'ultimo dovrà avere soltanto un neurone e funzione di attivazione `sigmoid`.
5. Compilare il modello di rete neurale usando come funzione di costo la `binary_crossentropy`, mentre come ottimizzatore l'algoritmo `sgd`. Si specifichi inoltre che il 30% dei dati dovrà essere usato per la validazione, che la metrica di riferimento sarà la `accuracy`, e che il numero di epoche per il training sarà pari a 50.

Commentare i risultati ottenuti.

> **Note**
> * la `shape` del layer di Input dovrà essere pari al numero di colonne di X;
> * X dovrà essere un oggetto di tipo `tf.Tensor`. Per ottenerlo, utilizzeremo il package `tensorflow.keras.backend`, importato all'inizio come `K`, con sintassi `output_array = K.constant(X)`;
> * se si utilizza un oggetto di tipo `ColumnTransformer`, è necessario convertire l'oggetto risultante da matrice sparsa Scipy ad array NumPy mediante il metodo [`toarray`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.toarray.html).

In [3]:
df = fetch_openml("titanic", version=1, as_frame=True).frame

df.drop(['name', 'ticket'], axis=1, inplace=True)
df['boat'] = df['boat'].apply(lambda x: '0' if x is None else x)
drop_nan(df)

df['survived'] = df['survived'].apply(lambda x: int(x))

y = df.pop('survived')

ct = ColumnTransformer(
    [('sex_tr', OneHotEncoder(handle_unknown='ignore'), ['sex']),
     ('embarked_tr', OneHotEncoder(handle_unknown='ignore'), ['embarked']),
     ('boat_tr', OneHotEncoder(handle_unknown='ignore'), ['boat']),
     ('home.dest_tr', OneHotEncoder(handle_unknown='ignore'), ['home.dest'])],
    remainder='passthrough')

X = ct.fit_transform(df).toarray()
X = K.constant(X)

In [4]:
model = keras.Sequential([
    layers.Input(shape=(383,)),
    layers.Dense(16, activation='relu', name='layer1'),
    layers.Dense(16, activation='relu', name='layer2'),
    layers.Dense(16, activation='relu', name='layer3'),
    layers.Dense(1, activation='sigmoid')
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 16)                6144      
_________________________________________________________________
layer2 (Dense)               (None, 16)                272       
_________________________________________________________________
layer3 (Dense)               (None, 16)                272       
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 6,705
Trainable params: 6,705
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)
model.fit(
    x=X,
    y=y,
    validation_split=0.3,
    epochs=50
)

Epoch 1/50
15/15 [==============================] - 2s 108ms/step - loss: 1.5999 - accuracy: 0.4952 - val_loss: 0.6417 - val_accuracy: 0.6311
Epoch 2/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6636 - accuracy: 0.6431 - val_loss: 0.6637 - val_accuracy: 0.6456
Epoch 3/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6463 - accuracy: 0.6324 - val_loss: 0.6432 - val_accuracy: 0.6359
Epoch 4/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6732 - accuracy: 0.6250 - val_loss: 0.6411 - val_accuracy: 0.6311
Epoch 5/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6533 - accuracy: 0.6441 - val_loss: 0.6604 - val_accuracy: 0.6117
Epoch 6/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6507 - accuracy: 0.6449 - val_loss: 0.6312 - val_accuracy: 0.6311
Epoch 7/50
15/15 [==============================] - 0s 4ms/step - loss: 0.6417 - accuracy: 0.6831 - val_loss: 0.6549 - val_accuracy: 0.6359
Epoch 8/50
15/15 [